In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Preview the facts table.

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [77]:
%%sql
SELECT MIN(population) AS minimum_population,
       MAX(population) AS maximum_population,
       MIN(population_growth) AS minimum_population_growth,
       MAX(population_growth) AS maximum_population_growth
  FROM facts;

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,7256490011,0.0,4.02


In [9]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [10]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [76]:
%%sql
SELECT MIN(population) AS minimum_population,
       MAX(population) AS maximum_population,
       MIN(population_growth) AS minimum_population_growth,
       MAX(population_growth) AS maximum_population_growth
  FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


minimum_population,maximum_population,minimum_population_growth,maximum_population_growth
0,1367485388,0.0,4.02


In [75]:
%%sql
SELECT AVG(population) AS average_population,
       AVG(area) AS average_area
  FROM facts
 WHERE name != 'World';

 * sqlite:///factbook.db
Done.


average_population,average_area
32242666.56846473,555093.546184739


In [15]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != 'World')
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name != 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Which country has the most people?een?

In [26]:
%%sql
SELECT *
  FROM facts
 WHERE name != 'World'
 ORDER BY population DESC
 LIMIT 1;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


Which country has the highest growth rate?

In [24]:
%%sql
SELECT *
  FROM facts
 WHERE name != 'World'
 ORDER BY population_growth DESC
 LIMIT 1;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


- Hmmm South Sudan doesn't have a correct value for area_land or area_water. We should remember that for later.

Which countries have the highest ratios of water to land?

In [79]:
%%sql
SELECT *, ROUND((area_water + 0.0) / area_land, 2) AS ratio_of_water_to_land
  FROM facts
 WHERE name != 'World'
 ORDER BY (area_water + 0.0) / area_land DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,ratio_of_water_to_land
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.67
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.52
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.55
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.39
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.28
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.26
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95,0.23
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.22
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0,0.16
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0,0.16


Which countries have more water than land?

In [74]:
%%sql
SELECT *, area_water - area_land AS water_land_diff
  FROM facts
 WHERE name != 'World'
   AND area_water > area_land
 ORDER BY area_water / area_land DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_land_diff
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,54280
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,1218


We know that there are rows without area_land and area_water values, let's at least surface those.
If need be we can have an intern go evalute those rows by hand.

In [73]:
%%sql
SELECT *
  FROM facts
 WHERE name != 'World'
   AND (area_water IS NULL OR area_land IS NULL);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22
126,nz,New Zealand,267710,267710,None,4438393,0.82,13.33,7.36,2.21
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
165,su,Sudan,1861484,None,None,36108853,1.72,29.19,7.66,4.29
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
210,fs,French Southern and Antarctic Lands,None,None,None,None,None,None,None,None
212,tb,Saint Barthelemy,None,None,None,7237,None,None,None,None
213,rn,Saint Martin,54,54,None,31754,None,None,None,None
225,ax,Akrotiri,123,None,None,15700,None,None,None,None


Which countries will add the most people to their populations next year?

In [89]:
%%sql
SELECT *, CAST(ROUND(population * (population_growth / 100)) AS INT) AS est_population_increase
  FROM facts
 WHERE name != 'World'
 ORDER BY population * (population_growth / 100) DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,est_population_increase
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04,15270686
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44,6153684
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22,4448270
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54,2906653
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22,2874562
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,2703324
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86,2506677
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16,2355142
40,cg,"Congo, Democratic Republic of the",2344858,2267048,77810,79375136,2.45,34.88,10.07,0.27,1944691
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09,1626074


Which countries have a higher death rate than birth rate?

In [95]:
%%sql
SELECT *
  FROM facts
 WHERE name != 'World'
   AND death_rate > birth_rate
 ORDER BY death_rate / birth_rate DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83
158,si,Slovenia,20273,20151,122,1983412,0.26,8.42,11.37,0.37
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
214,sb,Saint Pierre and Miquelon,242,242,0,5657,1.08,7.42,9.72,8.49


Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [113]:
%%sql
SELECT *, (population * 1.0) / area AS population_area_ratio
  FROM facts
 WHERE name != 'World'
 ORDER BY (population * 1.0) / area DESC
 LIMIT 20;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_area_ratio
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168.964285714286
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267.5
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141.279770444763
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445.041516245487
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191.819444444444
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876.333333333333
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771.8592105263158
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319.6409395973155
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310.01582278481
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299.9259259259259
